In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/FinalHack Datasets/train.csv",parse_dates = ['date'])


item_details = pd.read_excel("/content/drive/MyDrive/FinalHack Datasets/item_details.xlsx")

In [ ]:
train_2017_data = train[(train['date'] > '2016-12-31') & (train['date'] < '2017-08-16')]

train_2018_data = train[(train['date'] > '2017-12-31') & (train['date'] < '2018-08-16')]

In [ ]:
del train

In [ ]:
del train_2017_data
del train_2018_data

In [ ]:
train_subset.reset_index()

,index,date,locationId,item_id,unit_sales,onpromotion
0,38594264,2017-01-01,location_25,item_103665,12.0,False
1,38594265,2017-01-01,location_25,item_105575,23.0,False
2,38594266,2017-01-01,location_25,item_108634,1.0,False
3,38594267,2017-01-01,location_25,item_108698,6.0,False
4,38594268,2017-01-01,location_25,item_108786,6.0,False
...,...,...,...,...,...,...
37129246,88138654,2018-08-15,location_54,item_2026893,2.0,False
37129247,88138655,2018-08-15,location_54,item_2026945,1.0,False
37129248,88138656,2018-08-15,location_54,item_2026983,5.0,False
37129249,88138657,2018-08-15,location_54,item_2027090,2.0,False


In [ ]:
train_subset = pd.merge(train_subset,item_details,how = 'left', on = 'item_id')

In [ ]:
train_subset['class'].nunique()

319

In [ ]:
train_subset = pd.merge(train_subset,item_details,how = 'left', on = 'item_id')

train_subset['Month'] =pd.DatetimeIndex(train_subset['date']).month.astype('int8')
train_subset['Day'] =pd.DatetimeIndex(train_subset['date']).day.astype('int8')
train_subset['Week'] =pd.DatetimeIndex(train_subset['date']).weekday.astype('int8')

train_subset = train_subset.drop(['date','perishable','item_id'],axis = 1)


train_subset.head()

,locationId,unit_sales,onpromotion,category_of_item,class,Month,Day,Week
0,location_25,12.0,False,baked_items / bread_based,class_2712,1,1,6
1,location_25,23.0,False,grocery_items,class_1045,1,1,6
2,location_25,1.0,False,grocery_items,class_1075,1,1,6
3,location_25,6.0,False,deli_items,class_2644,1,1,6
4,location_25,6.0,False,cleaning_utilities,class_3044,1,1,6


In [ ]:
### Removing negative values in train data 

train_subset = train_subset[(train_subset['unit_sales']>0)]

In [ ]:
### Removing outliers 

Q1 = train_subset.unit_sales.quantile(0.25)
Q3 = train_subset.unit_sales.quantile(0.75)
print(Q1,Q3)


IQR = Q3 - Q1
print(IQR)

lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
print( lower_limit,upper_limit)


train_subset = train_subset[(train_subset.unit_sales < upper_limit)]

2.0 9.0
7.0
-8.5 19.5


In [ ]:
### Converting to category 


def cat_converter(df):
    for i in df.columns:
        if df[i].dtype == 'float64':
            df[i] = df[i]
        else:            
            df[i] = df[i].astype('category')
            print(i)
    return df


train_subset = cat_converter(train_subset)

locationId
onpromotion
category_of_item
class
Month
Day
Week


In [ ]:
cat_cols = ['locationId','onpromotion','category_of_item','class','Month','Day','Week']

In [ ]:
from sklearn.model_selection import train_test_split

X = train_subset.drop(['unit_sales'], axis = 1)

y = train_subset['unit_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
from lightgbm import LGBMRegressor 


## Light GBM 

LGB_model = LGBMRegressor(boosting_type= 'dart', 
                          num_leaves = 2**5 - 1,
                          objective = 'regression_l2',
                          max_depth = 8,
                          min_data_in_leaf = 50,
                          learning_rate = 0.01,
                          metric = 'l2')


LGB_model.fit(X_train, y_train_log,eval_set = (X_test,y_test_log),early_stopping_rounds = 50,categorical_feature = cat_cols,verbose = 0)

LGBMRegressor(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.01, max_depth=8,
              metric='l2', min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=50, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=31, objective='regression_l2',
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
LGB_Model_Y_train_pred = LGB_model.predict(X_train)
LGB_Model_Y_test_pred = LGB_model.predict(X_test)


LGB_Model_Y_train_pred =np.expm1(LGB_Model_Y_train_pred)
LGB_Model_Y_test_pred =np.expm1(LGB_Model_Y_test_pred)

Train_score_LGB= mean_absolute_percentage_error(y_train,LGB_Model_Y_train_pred)
Test_score_LGB = mean_absolute_percentage_error(y_test,LGB_Model_Y_test_pred)

print(Train_score_LGB)
print(Test_score_LGB)

58.521333997428634
58.519358967701365
